# Análise de conjunto de dados vindo do Spotify

## Projeto da disciplina SCC0252 - Visualização Computacional

### Membros

* Alexandre E. de Souza Jesus - alexandre_souza@usp.br - **12559506**

* Eduardo Zaffari Monteiro - eduardozaffarimonteiro@usp.br - **12559490**

* Pedro Henrique de Freitas Maçonetto - pedromaconetto@usp.br - **12675419**

In [1]:
import dash
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
import pandas as pd

import plotly.express as px
import numpy as np

from wordcloud import WordCloud
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Gráfico 1 - Bar Plot
Para o primeiro gráfico o modelo escolhido foi o barplot, esse modelo foi selecionado com objetivo de visualizar uma variável numérica (Aparecimento em playlists do Spotify) por um valor temporal (Anos). Para esses atributos seria interessante utilizar ou o gráfico de barras, como escolhido, ou gráfico de linha, porém o primeiro foi escolhido para apresentar uma terceira variável (Tonalidade). 

Dessa forma, o gráfico de barras empilhadas foi escolhido para dar melhor noção de proporção de tonalidade em cada ano, isso funciona melhor nesse gráfico do que um gráfico de várias linhas ou o gráfico de barras lado a lado.


## Gráfico 2 - 

## Gráfico 3 - Radar Plot
Para o gráfico multidimensional a escolha foi o Radar Plot. Esse tipo de gráfico é útil para exibir os dados com um número arbitrário de variáveis, e é recomendado para se comparar diversas variáveis quantitativas de uma só vez. Essa visualização foi escolhida pois há um bom número de atributos quantitativos no *dataset* que podem ser utilizados para explicar melhor qual é o perfil de cada artista no *Spotify*.

Dessa forma, ao escolher o artista do qual se deseja visualizar as informações, o usuário tem a possibilidade de entender melhor quais características fizeram com que aquele artista tenha se destacado em comparação aos demais. Vale ressaltar que os gráficos estão disponíveis apenas para os 15 artistas mais escutados na plataforma, o que permite que seja feita uma análise do porquê esses artistas ficaram tão populares. 

Os atributos escolhidos variam de quantidade total de *streams* até o *playlisting* (número de *playlists* em que o artista se encontra) e características médias de suas músicas, como energia e dançabilidade.

In [2]:
# Criando aplicativo
aplicativo = dash.Dash(__name__)

# Lendo os dados
df = pd.read_csv('https://raw.githubusercontent.com/EduardoDuX/DataViz/main/spotify-2023.csv', encoding='latin-1')
df_radar_plot = pd.read_csv('https://raw.githubusercontent.com/EduardoDuX/DataViz/main/media-artistas-principais.csv', index_col='main_artist')

# Tratamento dos dados
stream = df['streams'].apply(lambda x: x.isnumeric())
df = df[stream]
df['streams'] = df['streams'].astype('int64')

# Figura do grafico 1
fig_1 = px.bar(df,
             x='released_year',
             y='in_spotify_playlists',
             color='key',
             labels={
                     "in_spotify_playlists": "Playlists do Spotify",
                     "released_year": "Ano de lançamento",
                     "key": "Tonalidade"
                 })

# Figura do grafico 2
fig_2 = px.scatter(df, 
            size='artist_count',
            y='streams',
            x='in_spotify_playlists',
            color='artist_count',
            labels={
                     "streams": "Quantidade de reproduções",
                     "in_spotify_playlists": "Playlists em que aparece",
                     "artist_count": "Artistas envolvidos"
                 })

# Figura do gráfico 3
fig_3 = px.line_polar(df_radar_plot, df_radar_plot.iloc[0], theta=df_radar_plot.columns, line_close=True)
fig_3.update_traces(fill='toself')

aplicativo.layout = html.Div([

    # Titulo do projeto
    html.H1('Análise de dados do Spotify', style={'text-align':'center'}),
    

    # Gráfico 1
    html.Div([
        html.H1('Aparecimento em playlists do Spotify por ano de lançamento por tonalidade'),
        dcc.Graph(
            id='grafico_1',
            figure=fig_1
        ),
        html.H3('Anos no gráfico:'),
        dcc.RangeSlider(
            id='year-slider',
            min=df.released_year.min(),
            max=df.released_year.max(),
            step=3,
            value=[1980, 2023],
            marks={i: {'label': str(i), 'style': {'transform': 'rotate(-45deg)'}} 
                   for i in range(df.released_year.min(), df.released_year.max() + 1, 3)},
        )],
    style={'display': 'inline-block', 'width':'48%'}),


    # Gráfico 2

    # Gráfico 3
    html.Div([
        html.H1('Artista:'),
        dcc.Dropdown(df_radar_plot.index.tolist(), df_radar_plot.index[0], id='radar-dropdown'),
        html.H2('Atributos de artistas mais populares'),
        dcc.Graph(
            id='grafico_3',
            figure=fig_3
        )
    ],
    style={'display': 'inline-block','float': 'right', 'width':'48%'})
    


])

@aplicativo.callback(
    Output('grafico_1', 'figure'),
    Input('year-slider', 'value')
)
def update_grafico_1(year_list):
    df_updated = df[df['released_year'].isin(range(*year_list))]

    updated_fig = px.bar(df_updated,
             x='released_year',
             y='in_spotify_playlists',
             color='key',
             labels={
                     "in_spotify_playlists": "Playlists do Spotify",
                     "released_year": "Ano de lançamento",
                     "key": "Tonalidade"
                 })
    
    return updated_fig

@aplicativo.callback(
    Output('grafico_3', 'figure'),
    Input('radar-dropdown', 'value')
)
def update_grafico_3(artist):
    artists = df_radar_plot.index.tolist()
    index = artists.index(artist)

    updated_fig = px.line_polar(df_radar_plot, df_radar_plot.iloc[index], theta=df_radar_plot.columns, line_close=True)
    updated_fig.update_traces(fill='toself')

    return updated_fig


if __name__ == '__main__':
    aplicativo.run_server(debug=True)

C:\Users\Eduardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



C:\Users\Eduardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

